In [ ]:
# https://www.anthropic.com/engineering/building-effective-agents
# https://www.agentrecipes.com/

# https://www.arionkoder.com/blog/building-an-ai-powered-eligibility-system-for-us-healthcare-part-i
# https://www.arionkoder.com/blog/building-an-ai-powered-eligibility-system-for-us-healthcare-part-ii

## LLM Call

In [ ]:
from repenseai.genai.agent import Agent
from repenseai.genai.tasks.api import Task


agent = Agent(
    model="claude-3-7-sonnet-20250219", # "claude-3-5-sonnet-20241022"
    model_type="chat",
)

task = Task(
    user="What is the capital of france?",
    agent=agent,
    simple_response=True
)

response = task.run()

print()
print(response.replace(". ", ".\n"))

In [ ]:
# python fundamentos_ai/characters.py 0

## LLM Call - Output Estruturado

In [ ]:
from repenseai.genai.agent import Agent
from repenseai.genai.tasks.api import Task

from pydantic import BaseModel


class User(BaseModel):
    name: str
    age: int
    children: int


sample1 = """
My name is Samuel, I'm 36 years old and I have a 3-month-old daughter and I want to have 3 more kids.
"""

sample2 = """
You are talking to Karen. She is a 30-year-old woman who has been a teacher for the past 12 years.
She has two children, who are 12 and 15 years old.
"""

agent = Agent(
    model="gpt-4o",
    model_type="chat",
    json_mode=True,
    json_schema=User,
)

task = Task(
    user="Extract the data from the text: \n\n {user}",
    agent=agent,
    simple_response=True
)

response = task.run({"user": sample2})

print()
print(response)

## Workflows

In [ ]:
from repenseai.genai.agent import Agent
from repenseai.genai.tasks.api import Task
from repenseai.genai.tasks.workflow import Workflow

from pydantic import BaseModel


def check_children(context: dict):
    children = context["user_output"]["children"]
    return children > 0


class User(BaseModel):
    name: str
    age: int
    children: int


sample1 = """
My name is Samuel, I'm 36 years and I dont have kids.
"""

sample2 = """
You are talking to Karen. She is a 29-year-old woman who has been a teacher for the past 12 years.
She has two children, who are 12 and 15 years old.
"""

agent = Agent(
    model="gpt-4o",
    model_type="chat",
    json_mode=True,
    json_schema=User,
)

task = Task(
    user="Extract the data from the text: \n\n {user}",
    agent=agent,
    simple_response=True
)

workflow = Workflow(
    [
        [task, "user_output"],
        [check_children, "children_output"],
    ]
)
    
response = workflow.run({"user": sample2})

print()
print(response["children_output"])

## Workflow: Conditional Gate

In [ ]:
from repenseai.genai.agent import Agent
from repenseai.genai.tasks.api import Task
from repenseai.genai.tasks.workflow import Workflow
from repenseai.genai.tasks.conditional import BooleanConditionalTask
from repenseai.genai.tasks.function import FunctionTask

from pydantic import BaseModel


def check_children(context: dict):
    children = context["user_output"]["children"]
    return children > 0

def return_exit(context: dict):
    return "Exit"


class User(BaseModel):
    name: str
    age: int
    children: int


sample1 = """
My name is Samuel, I'm 36 years old and I dont have kids.
"""

sample2 = """
You are talking to Karen. She is a 29-year-old woman who has been a teacher for the past 12 years.
She has two children, who are 12 and 15 years old.
"""

agent = Agent(
    model="gpt-4o",
    model_type="chat",
    json_mode=True,
    json_schema=User,
)

agent2 = Agent(
    model="claude-3-7-sonnet-20250219",
    model_type="chat",
)

task = Task(
    user="Extract the data from the text: {user}",
    agent=agent,
    simple_response=True
)

task2 = Task(
    user="Create a sales pitch for {user_output} to travel to disneyland.",
    agent=agent2,
    simple_response=True
)

bool_task = BooleanConditionalTask(
    condition=check_children,
    true_task=task2,
    false_task=FunctionTask(return_exit),
)

workflow = Workflow(
    [
        [task, "user_output"],
        [bool_task, "sales_pitch"],
    ]
)
    
response = workflow.run({"user": sample1}) # "can you tell me a joke?"

print()
print(response["sales_pitch"])

## Agents

In [ ]:
from repenseai.genai.agent import Agent
from repenseai.genai.tasks.api import Task

def get_weather(latitude: float, longitude: float) -> str:
    """Get weather information for a location"""
    return "Sunny, 22°C"

def get_location(city: str) -> tuple:
    """Get coordinates for a city"""
    return (48.8566, 2.3522)  # Example for Paris

# Initialize agent with tools
agent = Agent(
    model="claude-3-7-sonnet-20250219",
    model_type="chat",
    tools=[get_weather, get_location]
)

# Create task
task = Task(
    user="can you tell me a joke?", # "can you tell me a joke?", "What's the weather like in Paris today?"
    agent=agent
)

response = task.run()

print()
print(response['response'])

In [ ]:
task.prompt

## MCP Servers

In [ ]:
# https://www.anthropic.com/news/model-context-protocol
# https://modelcontextprotocol.io/

# https://github.com/modelcontextprotocol/python-sdk (fastmcp - client sample)
# https://modelcontextprotocol.io/quickstart/client (chat loop code)

In [ ]:
import asyncio

import nest_asyncio
nest_asyncio.apply()

from repenseai.genai.mcp.server import Server
from repenseai.genai.agent import AsyncAgent
from repenseai.genai.tasks.api import AsyncTask

server = Server(
    name="teste_mcp", 
    command='python', 
    args=["servers/bmi.py"]
)

async def main():
    agent = AsyncAgent(
        model="claude-3-5-sonnet-20241022",
        model_type="chat",
        server=server
    )

    task = AsyncTask(
        user="qual o meu bmi? altura: {altura}, peso: {peso}",
        agent=agent
    )
    
    response = await task.run({"altura": "1,77", "peso": "95kg"})
    print("\n"+response['response'])

asyncio.run(main())

In [ ]:
# python fundamentos_ai/servers/memory.py

## RAG

In [ ]:
# https://brains.dev/2024/embeddings-medidas-de-distancia-e-similaridade/

# https://platform.openai.com/docs/guides/embeddings
# https://cohere.com/pt/rerank

In [ ]:
import os
import numpy as np

from openai import OpenAI

def cosine_similarity(embedding1, embedding2):
    dot_product = np.dot(embedding1, embedding2)

    norm1 = np.linalg.norm(embedding1)
    norm2 = np.linalg.norm(embedding2)

    similarity = dot_product / (norm1 * norm2)
    
    return similarity

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

text1 = "A filha do Samuel se chama Elisa"
text2 = "Os gatos do Samuel se chamam pelanca (gordinha), katika (pequena) e blue (medrosa)"

database = [text1, text2]
database_embeddings = client.embeddings.create(input=database, model="text-embedding-3-small")

query = "Qual o nome da gata gordinha do Samuel?"
query_embedding = client.embeddings.create(input=query, model="text-embedding-3-small").data[0].embedding

similarities = [cosine_similarity(query_embedding, doc_embedding.embedding) for doc_embedding in database_embeddings.data]
most_similar_index = np.argmax(similarities)

database_text = database[most_similar_index]

prompt = (
    "Você é um agente especializado em responder perguntas. "
    "Use a base de dados fornecida para responder melhorar sua resposta"
    f"Base de dados:\n\n{database_text}\n\n"
    f"Pergunta:\n\n {query}"
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "user", 
            "content": prompt
        }
    ], 
    model="gpt-4o"
)

response.choices[0].message.content


In [ ]:
database_text

In [ ]:
similarities

In [ ]:
database_embeddings.data